# آموزش فراخوانی موازی توابع با Phi-4 Mini و ONNX

این دفترچه نحوه استفاده از Phi-4 Mini با ONNX Runtime GenAI برای فراخوانی موازی توابع را نشان می‌دهد. فراخوانی توابع به مدل اجازه می‌دهد تا ابزارها و APIهای خارجی را به صورت هوشمندانه بر اساس درخواست‌های کاربر فراخوانی کند.

## مرور کلی

در این آموزش، یاد خواهید گرفت که:
- Phi-4 Mini را با ONNX Runtime GenAI تنظیم کنید
- طرح‌های توابع برای رزرو پرواز و هتل تعریف کنید
- از تولید هدایت‌شده با دستور زبان Lark برای خروجی ساختاریافته استفاده کنید
- فراخوانی موازی توابع را برای سناریوهای پیچیده رزرو سفر اجرا کنید

## پیش‌نیازها

قبل از اجرای این دفترچه، مطمئن شوید که:
- مدل Phi-4 Mini ONNX را دانلود کرده‌اید
- بسته `onnxruntime-genai` را نصب کرده‌اید
- درک پایه‌ای از مفاهیم فراخوانی توابع دارید


## مرحله ۱: وارد کردن کتابخانه‌های مورد نیاز

ابتدا کتابخانه‌های لازم برای پیاده‌سازی فراخوانی تابع را وارد می‌کنیم.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## مرحله ۲: تنظیم و پیکربندی مدل

حالا مدل Phi-4 Mini ONNX را پیکربندی می‌کنیم. مطمئن شوید که مسیر را با دایرکتوری واقعی مدل خود جایگزین کنید.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## مرحله ۳: تنظیم پارامترهای تولید

پارامترهای تولید را تنظیم کنید تا رفتار خروجی مدل کنترل شود. این تنظیمات پاسخ‌های دقیق و متمرکز برای فراخوانی توابع را تضمین می‌کنند.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## مرحله ۴: تعریف توابع موجود

در این مرحله، توابعی که دستیار هوش مصنوعی ما می‌تواند فراخوانی کند را تعریف می‌کنیم. در این مثال، دو تابع مرتبط با سفر داریم:
1. **booking_flight_tickets**: برای رزرو پرواز بین فرودگاه‌ها
2. **booking_hotels**: برای رزرو اقامت در هتل‌ها

تعریف توابع مطابق با قالب طرح فراخوانی توابع OpenAI انجام می‌شود.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## مرحله ۵: توابع کمکی تولید گرامر

این توابع کمکی تعریف‌های تابع ما را به قالب گرامر Lark تبدیل می‌کنند که توسط ONNX Runtime GenAI برای تولید هدایت‌شده استفاده می‌شود. این کار تضمین می‌کند که مدل خروجی‌های معتبر از فراخوانی توابع را در قالب JSON صحیح تولید کند.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## مرحله ۶: آزمایش تولید گرامر

بیایید عملکردهای تولید گرامر خود را آزمایش کنیم تا ببینیم چگونه تعریف ابزارهای ما را به قالب مناسب تبدیل می‌کنند.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## مرحله ۷: آماده‌سازی پیام سیستم و تولیدکننده

حالا پیام سیستم را ایجاد می‌کنیم که مدل را درباره ابزارهای موجود مطلع می‌کند و تولیدکننده را با پارامترهای تولید هدایت‌شده تنظیم می‌کنیم.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## مرحله ۸: مقداردهی اولیه به تولیدکننده با تولید هدایت‌شده

حالا تولیدکننده را با پارامترهای تنظیم‌شده خود ایجاد می‌کنیم و دستور زبان Lark را برای تولید هدایت‌شده اعمال می‌کنیم.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## مرحله ۹: آزمایش فراخوانی موازی توابع

حالا بیایید تنظیمات خود را با یک سناریوی پیچیده رزرو سفر که نیاز به فراخوانی چندین تابع دارد، آزمایش کنیم.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### تولید فراخوانی‌های تابع

مدل اکنون فراخوانی‌های تابع ساختاریافته‌ای را بر اساس درخواست کاربر تولید خواهد کرد. به لطف تولید هدایت‌شده، خروجی در قالب JSON معتبر خواهد بود که می‌توان آن را مستقیماً اجرا کرد.

**خروجی مورد انتظار**: مدل باید فراخوانی‌های تابع زیر را تولید کند:
1. `booking_flight_tickets` با جزئیات پرواز از پکن (PEK) به پاریس (CDG)
2. `booking_hotels` با جزئیات اقامت در پاریس

سلول زیر را اجرا کنید تا تولید زنده را مشاهده کنید:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## نتیجه‌گیری

🎉 **تبریک!** شما با موفقیت فراخوانی موازی توابع با Phi-4 Mini و استفاده از ONNX Runtime GenAI را پیاده‌سازی کردید.

### آنچه آموختید:

1. **تنظیم مدل**: نحوه پیکربندی Phi-4 Mini با ONNX Runtime GenAI  
2. **تعریف توابع**: نحوه تعریف ساختارهای توابع برای فراخوانی توابع هوش مصنوعی  
3. **تولید هدایت‌شده**: نحوه استفاده از دستور زبان Lark برای تولید خروجی ساختاریافته  
4. **فراخوانی موازی توابع**: نحوه مدیریت درخواست‌های پیچیده که نیاز به چندین فراخوانی توابع دارند  

### مزایای کلیدی:

- ✅ **خروجی ساختاریافته**: تولید هدایت‌شده، فراخوانی توابع JSON معتبر را تضمین می‌کند  
- ✅ **پردازش موازی**: مدیریت چندین فراخوانی توابع در یک درخواست  
- ✅ **عملکرد بالا**: ONNX Runtime عملکرد بهینه‌ای برای استنتاج فراهم می‌کند  
- ✅ **ساختار منعطف**: امکان افزودن یا تغییر آسان تعریف توابع  

### منابع:

- [مستندات Phi-4 Mini](https://huggingface.co/microsoft/Phi-4-mini-onnx)  
- [مستندات ONNX Runtime GenAI](https://onnxruntime.ai/docs/genai/)  
- [بهترین روش‌های فراخوانی توابع](https://platform.openai.com/docs/guides/function-calling)  



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
